# New Section

In [22]:
%matplotlib
import matplotlib.pyplot as plt

import sys
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()


# S3 bucket name
bucket = 'sagemaker-us-west-2-479862333671'
subfolder = ''
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']


Using matplotlib backend: agg


In [23]:
model_data='s3://sagemaker-us-west-2-479862333671/perdictions/output/sagemaker-xgboost-2022-01-22-10-50-25-985/output/model.tar.gz'

In [24]:
model_data

's3://sagemaker-us-west-2-479862333671/perdictions/output/sagemaker-xgboost-2022-01-22-10-50-25-985/output/model.tar.gz'

In [25]:
from sagemaker import image_uris


In [26]:
image=image_uris.retrieve(framework='xgboost',region='us-west-2',version='1.2-1')

In [27]:
image 

'246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.2-1'

In [28]:
role

'arn:aws:iam::479862333671:role/service-role/AmazonSageMaker-ExecutionRole-20220103T082598'

In [29]:
trainedmodel = sagemaker.model.Model(
    image_uri=image,
    model_data = model_data,
    role=role)

In [30]:
model_transformer = trainedmodel.transformer(instance_count=1, instance_type='ml.c5.4xlarge')

In [31]:
test_location = 's3://{}/{}'.format(bucket, 'data/test_no_header.csv')

In [ ]:
model_transformer.transform(test_location, content_type='text/csv', split_type='Line')

....

In [ ]:
model_transformer.wait()

In [ ]:
!aws s3 cp --recursive $model_transformer.output_path $data_dir

In [2]:
data_dir = ''


In [3]:
Y_pred = pd.read_csv('s3://{}/{}'.format(bucket, 'data/X_val_values.csv.out'), header=None)
Y_pred.columns = ['demand']

In [4]:
Y_pred

,demand
0,0.471162
1,0.517790
2,0.536669
3,0.517790
4,0.471162
...,...
823225,1.077206
823226,0.504309
823227,1.342117
823228,1.404573


In [5]:
Y_test = pd.read_csv('s3://{}/{}'.format(bucket, 'data/val.csv'))['demand'].to_frame()

In [6]:
Y_test

,demand
0,0
1,0
2,0
3,2
4,0
...,...
823225,1
823226,1
823227,0
823228,1


In [23]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mean_squared_error(Y_test, Y_pred, squared=True)


6.131172588601535

In [24]:
mean_squared_error(Y_test, Y_pred, squared=False)

2.476120471342526

In [26]:
mean_absolute_error(Y_test, Y_pred)

1.2007927503676232